In [10]:
from pathlib import Path

import maliput

In [13]:
RESOURCE_PATH = Path(".") / ".." / "src" / "maliput_malidrive" / "maliput_malidrive" / "resources"
XODR_PATH = RESOURCE_PATH / "figure8_trafficlights" / "figure8_trafficlights.xodr"
YAML_PATH = RESOURCE_PATH / "figure8_trafficlights" / "figure8_trafficlights.yaml"
assert XODR_PATH.exists()
assert YAML_PATH.exists()

In [ ]:
# Load the network
road_network_options = {
    "opendrive_file": str(XODR_PATH.absolute()),
    "road_rule_book": str(YAML_PATH.absolute())
}
road_network_options["traffic_light_book"] = road_network_options["road_rule_book"]
road_network_options["phase_ring_book"] = road_network_options["road_rule_book"]
road_network_options["intersection_book"] = road_network_options["road_rule_book"]
network = maliput.plugin.create_road_network_from_plugin("maliput_malidrive", road_network_options)

In [64]:
# A compact example of getting the direction usage rule, speed limit, and vehicle usage of a position
def find_relevant_rules(network, p_inertial):
    
    # First, find the rules we care about. 
    speed_limit_hint = "Speed Limit"
    direction_usage_hint = "Direction Usage"
    vehicle_usage_hint = "Vehicle Usage"
    speed_limit_type = None
    vehicle_usage_type = None
    direction_usage_type = None
    for rule_type in network.rule_registry().DiscreteValueRuleTypes():
        if rule_type.string().find(direction_usage_hint) >= 0:
            direction_usage_type = rule_type
        elif rule_type.string().find(vehicle_usage_hint) >= 0:
            vehicle_usage_type = rule_type
    
    for rule_type in network.rule_registry().RangeValueRuleTypes():
        if rule_type.string().find(speed_limit_hint) >= 0:
            speed_limit_type = rule_type
    
    assert speed_limit_type is not None
    assert direction_usage_type is not None
    assert vehicle_usage_type is not None
    
    # Find the lane position for the inertial position.
    p_road_result = network.road_geometry().ToRoadPosition(p_inertial)
    assert p_road_result.distance < 1.0
    p_road = p_road_result.road_position
    
    # Query for rules.
    query_ranges = [
        maliput.api.LaneSRange(
            p_road.lane.id(),
            maliput.api.SRange(
                p_road.pos.s(),
                p_road.pos.s(),
            )
        )
    ]
    relevant_rules = network.rulebook().FindRules(query_ranges, 0.01)
    
    # Populate the result
    result = {
        "speed_limit": None,
        "direction_usage": None,
        "vehicle_usage": None
    }
    
    for rule_id in relevant_rules.discrete_value_rules:
        rule = relevant_rules.discrete_value_rules[rule_id]
        state_result = network.discrete_value_rule_state_provider().GetState(rule.id())
        if rule.type_id() == direction_usage_type:
            result["direction_usage"] = [rule, state_result.state]
        elif rule.type_id() == vehicle_usage_type:
            result["vehicle_usage"] = [rule, state_result.state]
    
    for rule_id in relevant_rules.range_value_rules:
        rule = relevant_rules.range_value_rules[rule_id]
        state_result = network.range_value_rule_state_provider().GetState(rule.id())
        if rule.type_id() == speed_limit_type:
            result["speed_limit"] = [rule, state_result.state]
    
    print("States")
    print("---")
    print("DirectionUsage id: {}, State: {}, Severity: {}".format(
        result["direction_usage"][0].id().string(),
        result["direction_usage"][1].value,
        result["direction_usage"][1].severity,
    ))
    print("VehicleUsage id: {}, State: {}, Severity: {}".format(
        result["vehicle_usage"][0].id().string(),
        result["vehicle_usage"][1].value,
        result["vehicle_usage"][1].severity,
    ))
    print("SpeedLimit id: {}, Min/Max: {}/{}, Severity: {}".format(
        result["speed_limit"][0].id().string(),
        result["speed_limit"][1].min,
        result["speed_limit"][1].max,
        result["speed_limit"][1].severity,
    ))
    
        



In [65]:
p_inertial = maliput.api.InertialPosition(5,0,0)
find_relevant_rules(network, p_inertial)

States
---
DirectionUsage id: Direction Usage Rule Type/96_3_-1, State: WithS, Severity: 0
VehicleUsage id: Vehicle Usage Rule Type/96_3_-1, State: NonPedestrians, Severity: 0
SpeedLimit id: Speed Limit Rule Type/96_3_-1_1, Min/Max: 0.0/11.11111111111111, Severity: 0
